In [1]:
import argparse
import os
import torch.backends.cudnn as cudnn
import torch
from torch import nn
from torch.nn import functional as F
import tqdm
import numpy as np
from PIL import Image
from models.sdafnet import SDAFNet_Tryon
from torchvision import transforms, utils
from utils import lpips
from utils.utils import AverageMeter
from torchvision import transforms, utils
from torch.utils import data
from torchvision.utils import save_image
from datasets import VITONDataset

In [2]:
sdafnet = SDAFNet_Tryon(ref_in_channel=6).to("mps")
sdafnet = sdafnet.to("mps")
sdafnet.load_state_dict(torch.load("ckpt_viton.pt", map_location='mps'))
sdafnet.eval()

SDAFNet_Tryon(
  (source_features): FeatureEncoder(
    (encoders): ModuleList(
      (0): Sequential(
        (0): DownSample(
          (block): Sequential(
            (0): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (1): ReLU(inplace=True)
            (2): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          )
        )
        (1): ResBlock(
          (block): Sequential(
            (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (1): ReLU(inplace=True)
            (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (4): ReLU(inplace=True)
            (5): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          )
        )
        (2): ResBlock(
          (block): Sequential(
       

In [3]:
def test(opt, net):
    test_dataset = VITONDataset(opt)
    test_loader = data.DataLoader(test_dataset, batch_size=opt.batch_size, shuffle=opt.shuffle,num_workers=opt.workers)
    with torch.no_grad():
        for i, inputs in enumerate(tqdm.tqdm(test_loader)):
            img_names = inputs['img_name']
            cloth_names = inputs['c_name']['paired']
            img = inputs['img'].to('mps')
            img_agnostic = inputs['img_agnostic'].to('mps') #Masked model image
            pose = inputs['pose'].to('mps')
            cloth_img = inputs['cloth']['paired'].to('mps')
            img =  F.interpolate(img, size=(256, 192), mode='bilinear')
            cloth_img = F.interpolate(cloth_img, size=(256, 192), mode='bilinear')
            img_agnostic = F.interpolate(img_agnostic, size=(256, 192), mode='bilinear')
            pose = F.interpolate(pose, size=(256, 192), mode='bilinear')
            ref_input = torch.cat((pose, img_agnostic), dim=1)
            tryon_result = net(ref_input, cloth_img, img_agnostic).detach()
            for j in range(tryon_result.shape[0]):
                save_image(tryon_result[j:j+1], os.path.join(opt.save_dir, opt.name, "vis_viton_out", img_names[j]), nrow=1, normalize=True, range=(-1,1))
            if opt.add_compare:
                tryon_result = torch.cat([img_agnostic, cloth_img, tryon_result],2)
                save_image(tryon_result, os.path.join(opt.save_dir, opt.name, "vis_viton_out", img_names[0]), nrow=10, normalize=True, range=(-1,1))

In [4]:
def get_opt():
    parser = argparse.ArgumentParser()
    parser.add_argument('--name', type=str, default="TEST_PAIR")
    parser.add_argument('-b', '--batch_size', type=int, default=4)
    parser.add_argument('--lr', type=float, default=0.0004)
    parser.add_argument('-j', '--workers', type=int, default=4)
    parser.add_argument('--load_height', type=int, default=256)
    parser.add_argument('--load_width', type=int, default = 192)
    parser.add_argument('--mode', type=str, default='test')
    parser.add_argument('--shuffle', action='store_true')
    parser.add_argument('--add_compare', action='store_true')
    parser.add_argument('--dataset_dir', type=str, default='./test_data')
    parser.add_argument('--dataset_imgpath', type=str, default='data')
    parser.add_argument('--dataset_list', type=str, default='test_pairs.txt')
    parser.add_argument('--checkpoint_dir', type=str, default='./checkpoints/')
    parser.add_argument('--save_dir', type=str, default='./results/')
    parser.add_argument('--multi_flows', type=int, default=6)
    parser.add_argument('--display_freq', type=int, default=100)
    opt = parser.parse_args(args=[])
    return opt
opt = get_opt()
print(opt)

Namespace(name='TEST_PAIR', batch_size=4, lr=0.0004, workers=4, load_height=256, load_width=192, mode='test', shuffle=False, add_compare=False, dataset_dir='./test_data', dataset_imgpath='data', dataset_list='test_pairs.txt', checkpoint_dir='./checkpoints/', save_dir='./results/', multi_flows=6, display_freq=100)


In [5]:
test(opt, sdafnet)

  0%|          | 0/1 [00:00<?, ?it/s]/opt/homebrew/Caskroom/miniconda/base/envs/gpu/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/Users/shimhamin/Documents/GitHub/ai-fitting-room/models/sdafnet.py:118: UserWarning: MPS: no support for int64 repeats mask, casting it to int32 (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/mps/operations/Repeat.mm:236.)
  att_maps = torch.repeat_interleave(att_maps, out_ch, 1)
/opt/homebrew/Caskroom/miniconda/base/envs/gpu/lib/python3.9/site-packages/torch/nn/functional.py:4236: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the o